# <center> Are there zeros in the magnetic field from a solenoid? </center> 
## <center> A computational essay for the subject FYS1120, semester H2024 </center>
<hr>

<center>Authors: Hai Duc Pham, John Isaac Calderon</center>

## Innledning
I denne computational essayen for FYS1120 skal vi utdype oss, prinsippet bak samt med historien rundt Solenoiden. Videre skal vi se på hovedbudskapet om at en solenoide faktisk har null felt, både det magnetiske og det elektriske feltet i systemet, ved å visualisere og tilpasse variablene. Vi skal utdype metoden samt fremgangsmåten for hvordan vi skal finne ut av problemstillingen til prosjektet vårt. Dette arbeidet vil foregå i samarbeid med en annen, men dokumentasjonen og kodingen vil være hensiktsmessig tilpasset min forståelse av oppgaven. Dette gjelder også utviklingen av KI i nåværende samfunn, der det brukes til å dobbeltsjekke faktiske opplysninger. Det ble __brukt grammatiske KI-verktøy__ for å minske feil i grammatikken eller budskapet i essayen.

## 2.1 Hva er en Solenoide?

Solenoiden er en isolert leder bygget som en tettviklet spole. Når strøm tilføres systemet, vil den danne et elektromagnetisk felt rundt solenoiden. Når strøm sendes gjennom spolen, vil magnetfeltet bevege seg gjennom spolen og langs aksen til solenoiden. Dette kan gi mulighet å overføre den elektirske energien til en mekansike energi ved den mangetiske feltet. Det finnes 2 foskjellige soleniode som ekstierer i markende det enten "valve eller electric". Begge i prinsippe har egenskapen ved å dytte eller hente materialen den er polet til Solenoiden. Komponenten kan brukes i en rekke forskjellige systemer, det er mulig å bruke solenoide som endringer mellom forskjellige spenninger ved bygging av transformatorer og generatorer for generering av strøm.

## 2.2 Fremgangsmåte
Det er rekke fremgangsmåte som kan simuleres en visuelt Solenoide. I denne Computatinoal Essay skal det gjøres i Pythonm med tre forskjellige planer : XY, YZ og XZ planet. Det skal også trengs beregninger som kan gis ut magnetiske feltet rundt solenoide. Etterkan kjører vi et seperat program der den leter etter en mørk / død sone ved at $B = 0$ i alle tre planene.

Det er to hoved formeler som blir burkt i denne prosjektet den elektriske feltet $\mathbf{E}$ og det magnetiske feltet $\mathbf{B}$: 

#### Elektrisk felt $\mathbf{E}$

1. **Elektrisk felt fra en punktladning**:
   $$ \mathbf{E} = \frac{1}{4\pi\varepsilon_0} \frac{Q}{r^2} \hat{\mathbf{r}} $$
   hvor:
   - $ Q $ er ladningen,
   - $ r $ er avstanden fra ladningen,
   - $ \hat{\mathbf{r}} $ er enhetsvektoren fra ladningen til punktet,
   - $\varepsilon_0 $ er vakuumpermittiviteten.

2. **Forholdet til elektrisk potensial**:
   $
   \mathbf{E} = -\nabla V
   $
   hvor $V $ er det elektriske potensialet.

#### Magnetisk felt $\mathbf{B}$

1. **Magnetisk felt fra en strømførende leder (siden vi ser fra Soleonide bevegelse) (Biot-Savarts lov)**:
   $$
   \mathbf{B} = \frac{\mu_0}{4\pi} \int \frac{I \, d\mathbf{l} \times \hat{\mathbf{r}}}{r^2}
   $$
   hvor:
   - $I$ er strømmen,
   - $d\mathbf{l}$ er differensial lengdeelement av lederen,
   - $\mathbf{\hat{r}}$ er enhetsvektoren fra elementet til punktet,
   - $\mu_0$ er vakuumpermeabiliteten.

2. **Forholdet til vektorpotensialet**:
   $
   \mathbf{B} = \nabla \times \mathbf{A}
   $
   hvor $ \mathbf{A} $ er det magnetiske vektorpotensialet.



"Gi beskjed hvis du trenger mer detaljer eller spesifikke formler!" - Hai 

### 2.3 Koding til planene

### 3.1 Endringer i størrelse solniode kan skape null B-felt??
Det ble sett gjennom hvordan det var i helt standardisert størrelse. Men vil det få en null magnetfelt hvis vi gjør den større eller mindre på solenoiden? Hvis vi utdyper oss den formelen vi har brukt i for mangetiske feltet 
    $$
   \mathbf{B} = \frac{\mu_0}{4\pi} \int \frac{I \, d\mathbf{l} \times \hat{\mathbf{r}}}{r^2}
    $$
Vil det tar hensyn med lengden til over tid. Skal vi bruke den animasjon plotting der det kan ses forksjeller i B feltet i xy og xz (ses at det ikke endringer blir det ses to planer for mindre kjøring i maskinen) 
    (skriv mer her - Hai)


### 3.2 Koding til variert størrelse

### 4.1 Serie og paraell?
Ettersom at solenoiden i normal og variert skapte ---- ( skriv resultatet her). Er det mulighet lage da serie og paraellekoblinger til null B felt? 

### 4.2 koding til serie og paraelle.

### 5.1 Konklusjon eller sammendrag
Fra det vi har funnet om det er mulig at det finnes null $\mathbf{B}$- felt i hele systemet så ble det oppdaget at det ---- ( skriv svaret her)

De rekke mulige feilkilder som kan medfører usikkheter eller kritiske påvirkninger til simuleringer vi har funnet. Det settes som sjekkliste under

## Kildeføring
- Fakta om Solenoide - https://snl.no/solenoid
- Utvidelse av Soleniode - https://ccoils.com/blog/what-is-a-solenoid/ 